In [1]:
import pandas as pd
import numpy as np
import folium

In [2]:
df_delay=pd.read_csv('data_map.csv')

In [3]:
df_cities=pd.read_csv('brazil_cities.csv')

In [4]:
df_cities

,city,lat,lon,country,iso2,admin,capital,population,population_proper
0,São Paulo,-23.573252,-46.641681,Brazil,BR,São Paulo,admin,18845000.0,10021295.0
1,Rio de Janeiro,-22.875113,-43.277548,Brazil,BR,Rio de Janeiro,admin,11748000.0,2010175.0
2,Belo Horizonte,-19.901739,-43.964196,Brazil,BR,Minas Gerais,admin,5575000.0,2373224.0
3,Porto Alegre,-30.033333,-51.200000,Brazil,BR,Rio Grande do Sul,admin,3917000.0,1372741.0
4,Brasília,-15.783333,-47.916667,Brazil,BR,Distrito Federal,primary,3716996.0,2562963.0
...,...,...,...,...,...,...,...,...,...
382,Ribeirão Pires,-23.693784,-46.466229,NaN,NaN,NaN,NaN,NaN,NaN
383,Baixada Santista,-24.043272,-46.738282,NaN,NaN,NaN,NaN,NaN,NaN
384,Cubatão,-23.867970,-46.479179,NaN,NaN,NaN,NaN,NaN,NaN
385,Itaquaquecetuba,-23.464068,-46.369003,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
matches=list(df_delay.mesoregion[df_delay.mesoregion.isin(df_cities.city)])
matches

['Baixada Santista',
 'Belo Horizonte',
 'Brasília',
 'Campinas',
 'Cubatão',
 'Curitiba',
 'Florianópolis',
 'Fortaleza',
 'Goiânia',
 'Itaquaquecetuba',
 'Joinville',
 'Manaus',
 'Piracicaba',
 'Porto Alegre',
 'Recife',
 'Ribeirão Pires',
 'Ribeirão Preto',
 'Rio de Janeiro',
 'Salvador',
 'Sorocaba',
 'São José dos Campos',
 'São Paulo',
 'Uberlândia',
 'Vitória']

In [6]:
missing=[x for x in df_delay.mesoregion if x not in matches]

In [7]:
missing

[]

Create a dictionary for matching cities

In [8]:
dict_cities = {key:value for (key,value) in  zip(matches,matches)}

In [9]:
dict_cities

{'Baixada Santista': 'Baixada Santista',
 'Belo Horizonte': 'Belo Horizonte',
 'Brasília': 'Brasília',
 'Campinas': 'Campinas',
 'Cubatão': 'Cubatão',
 'Curitiba': 'Curitiba',
 'Florianópolis': 'Florianópolis',
 'Fortaleza': 'Fortaleza',
 'Goiânia': 'Goiânia',
 'Itaquaquecetuba': 'Itaquaquecetuba',
 'Joinville': 'Joinville',
 'Manaus': 'Manaus',
 'Piracicaba': 'Piracicaba',
 'Porto Alegre': 'Porto Alegre',
 'Recife': 'Recife',
 'Ribeirão Pires': 'Ribeirão Pires',
 'Ribeirão Preto': 'Ribeirão Preto',
 'Rio de Janeiro': 'Rio de Janeiro',
 'Salvador': 'Salvador',
 'Sorocaba': 'Sorocaba',
 'São José dos Campos': 'São José dos Campos',
 'São Paulo': 'São Paulo',
 'Uberlândia': 'Uberlândia',
 'Vitória': 'Vitória'}

In [10]:
len(dict_cities)-len(df_delay)

0

In [11]:
?pd.merge

In [12]:
df_cities=df_cities.groupby(['city','lat','lon'])['country'].first().reset_index().iloc[:,:-1]

In [13]:
len(df_cities)

387

In [14]:
df_cities

,city,lat,lon
0,Abaetetuba,-1.724735,-48.881345
1,Abunã,-9.700000,-65.383333
2,Acaraú,-2.883333,-40.116667
3,Alagoinhas,-12.132504,-38.427789
4,Alenquer,-1.947293,-54.736185
...,...,...,...
382,Volta Redonda,-22.514005,-44.109594
383,Várzea Grande,-15.647000,-56.133000
384,Xinguara,-7.100569,-49.941751
385,Xique-Xique,-10.823803,-42.729650


In [15]:
df_delay=pd.merge(df_delay,df_cities,how='left',left_on='mesoregion',right_on='city')

In [16]:
df_delay.drop(columns=['city'],inplace=True)

In [17]:
df_delay.head(3)

,mesoregion,delay,package_id,lat,lon
0,Baixada Santista,2.107919,412,-24.043272,-46.738282
1,Belo Horizonte,4.062605,581,-19.901739,-43.964196
2,Brasília,4.544186,318,-15.783333,-47.916667


In [18]:
red_labels=['#ffffd4','#fe9929','#993404']
my_colors=pd.cut(df_delay.delay,3,labels=red_labels)

In [19]:
df_delay['colors']=my_colors

In [23]:
def createMap(data,col_text1,col_text2,col_radius,col_color):

    '''
    createMap uses CircleMaker method from folium. Inputs:

    - a dataframe
    - lat  and lon are included, with those names, in different columns/features 
    - Specify text column
    - Specify radius colums
    - Specify fill color column
    '''
   
    m=folium.Map([-9.2,-49.7],zoom_start=4)


    for lat,lon,lab1,lab2,radius,color in zip(data['lat'],
                                data['lon'],
                                data[col_text1],
                                data[col_text2],
                                data[col_radius],
                                data[col_color]):

        folium.CircleMarker([lat, lon],
                        popup="Mean Delay: "+str(round(lab1,1))+" days,\n" +str(lab2)+', '+
                            str(radius) +' delayed packages',
                        radius=30*np.log(radius)/np.log(12000),
                        color='black',
                        fill=True,
                        fill_opacity=0.7,
                        fill_color=color,
                       ).add_to(m)
        m.save('delay_map.html')

In [24]:
createMap(df_delay,'delay','mesoregion','package_id','colors')

In [25]:
df_delay.to_csv('data_map_final.csv')